# 5.1 인스타그램 크롤링 

### 5.1.2 인스타그램 접속 후 로그인하기

In [1]:
# 예제 5-1 chromedriver 열기
from selenium import webdriver

driver = webdriver.Chrome("./chromedriver.exe")   

#### 안내사항
로그인을 해야만 게시물을 볼 수 있도록 인스타그램 운영 정책이 변경되었습니다(2020.04.27)  
인스타그램 로그인을 하는 코드를 추가하였습니다.   
직접 브라우저에 계정 정보를 입력해도 되고, 코드를 이용해도 됩니다.   
다만,  본인의 계정/비번 정보가 외부에 노출되지 않도록 주의하시기 바랍니다. 

※ 주의) `인스타 로그인`을 위해 계정 `아이디` / `비번` 입력이 필요합니다. 

In [2]:
# 예제 5-2 인스타그램 접속하기
import time

# 인스타그램 접속하기
driver.get('https://www.instargram.com')
time.sleep(2)

---

#### ※ 로그인 페이지로 접속이 되지 않고 오류 화면이 보일 경우 아래 링크를 참고하세요
https://www.notion.so/playwithdata/c02e510507504b42ae6073c1fbb46f29

---

In [3]:
with open('password.txt') as fp:
    password = fp.read()

In [4]:
# 예제 5-3 인스타계정으로 로그인하기

email = 'ckiekim'   ### 계정 정보 수정 필요
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(3)

In [7]:
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    #time.sleep(1)
    #driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
except:
    pass

위 코드 실행후 인스타그램 로그인이 완료되어야 합니다.  

추가로, 2단계 인증이 필요하다고 나올 경우 인증까지 완료해주세요. 

인스타계정이 아닌 페이스북계정으로 로그인이 필요하거나  코드에 아이디/비번 입력하는 것이 부담되는 분은

수기로 직접 입력해서 로그인까지 완료하시면 됩니다. 



### 5.1.3 인스타그램 검색결과 URL 만들어서 접속하기

In [8]:
# 예제 5-4 검색결과 URL 만드는 함수
def insta_searching(word):
    
    url = 'https://www.instagram.com/explore/tags/' + word
    return url    

In [9]:
# 예제 5-5 검색결과 페이지 접속하기
word = "제주도맛집"
url = insta_searching(word)
driver.get(url)

### 5.1.4 첫 번째 게시글 열기 

In [10]:
# 예제 5-6 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

### 5.1.5 게시글 정보 가져오기 

In [11]:
# 예제 5-7 게시글 정보 가져오기
import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content) 
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

['(협찬)애월바다와 선셋🌅감성까지 더 한스페셜 카레맛집!😋다양한 카레와 토핑들로특별함을 UP!#사이카레@saicurry_aewolOPEN 10:00CLOSE 20:00🙅\u200d♀️연중무휴(임시휴무 인스타공지)🙅NO브레이크타임🙆재료소진시 조기마감될 수 있어요📞064)752.4279위치 - 제주시 애월읍 애월해안로 752 2층  (구엄리 509)#사이카레 #구엄리맛집 #애월맛집 #제주맛집 #제주도맛집#애월밥집 #제주공항근처맛집 #애월카페#애월가볼만한곳 #협재맛집 #한림맛집#제주도민맛집 #제주오션뷰 #제주시맛집#애월해안도로맛집 #한담맛집 #한담카페#협재카페 #한림카페 #제주카페#제주애월맛집 #애월핫플레이스 #제주핫플',
 '2021-08-17',
 0,
 '',
 ['#사이카레@saicurry_aewolOPEN',
  '#사이카레',
  '#구엄리맛집',
  '#애월맛집',
  '#제주맛집',
  '#제주도맛집',
  '#애월밥집',
  '#제주공항근처맛집',
  '#애월카페',
  '#애월가볼만한곳',
  '#협재맛집',
  '#한림맛집',
  '#제주도민맛집',
  '#제주오션뷰',
  '#제주시맛집',
  '#애월해안도로맛집',
  '#한담맛집',
  '#한담카페',
  '#협재카페',
  '#한림카페',
  '#제주카페',
  '#제주애월맛집',
  '#애월핫플레이스',
  '#제주핫플']]

### 5.1.6 다음 게시글 열기

In [12]:
# 예제 5-8 다음 게시글 열기
def move_next(driver):

    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

move_next(driver)

In [14]:
driver.find_element_by_css_selector('button.wpO6b').click()

### 5.1.7여러 게시글 정보 수집하기

In [16]:
# 예제 5-10 인스타그램 크롤링  ②
# ② 인스타그램 검색페이지 URL 만들기
word = "제주도맛집"    #검색어
url = insta_searching(word)

# ③ 검색페이지 접속하기
driver.get(url)
time.sleep(5)

# ④ 첫 번째 게시글 열기
select_first(driver)

In [17]:
# ⑤ 비어있는 변수(results)만들기
results = [ ]

# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 50      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(3)
        move_next(driver)
    

print(results[:2])

[['<광고>제주갈비국수가 유명한 맛집!!손님들을 위해 끝까지 최선을 다해 드실수 있도록!!! 신경쓰는 #산도롱맨도롱▪️오픈시간: 07:30 - 17:00▪️전화: 064-782-5105▪️제주시 구좌읍 해맞이해안로 2284▪️매주 화요일 휴무렌터카네비는 가끔 틀리실수 있으니 폰네비로 찾아오세요~~~ ✴️ 07:30~10:00 여유롭게 식사가 가능합니다.#성산일출봉맛집 #성산아침식사 #성산맛집#제주맛집 #제주여행 #종달리 #종달리맛집#우도여행 #성산항 #성산항맛집 #종달항#제주도맛집 #우도가는길 #우도여행코스 #우도#성산일출봉근처맛집 #종달리수국길 #우도맛집', '2021-08-16', 0, '', ['#산도롱맨도롱▪️오픈시간:', '#성산일출봉맛집', '#성산아침식사', '#성산맛집', '#제주맛집', '#제주여행', '#종달리', '#종달리맛집', '#우도여행', '#성산항', '#성산항맛집', '#종달항', '#제주도맛집', '#우도가는길', '#우도여행코스', '#우도', '#성산일출봉근처맛집', '#종달리수국길', '#우도맛집']], ['(협찬)애월바다와 선셋🌅감성까지 더 한스페셜 카레맛집!😋다양한 카레와 토핑들로특별함을 UP!#사이카레@saicurry_aewolOPEN 10:00CLOSE 20:00🙅\u200d♀️연중무휴(임시휴무 인스타공지)🙅NO브레이크타임🙆재료소진시 조기마감될 수 있어요📞064)752.4279위치 - 제주시 애월읍 애월해안로 752 2층  (구엄리 509)#사이카레 #구엄리맛집 #애월맛집 #제주맛집 #제주도맛집#애월밥집 #제주공항근처맛집 #애월카페#애월가볼만한곳 #협재맛집 #한림맛집#제주도민맛집 #제주오션뷰 #제주시맛집#애월해안도로맛집 #한담맛집 #한담카페#협재카페 #한림카페 #제주카페#제주애월맛집 #애월핫플레이스 #제주핫플', '2021-08-17', 0, '', ['#사이카레@saicurry_aewolOPEN', '#사이카레', '#구엄리맛집', '#애월맛집', '#제주맛집', '#제주도맛집', '#애월밥집', '#제주공항근처

In [20]:
# 예제 5-11 크롤링 결과 저장하기

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_csv('제주도맛집.csv', index=False)

In [19]:
results_df.head()

,content,data,like,place,tags
0,<광고>제주갈비국수가 유명한 맛집!!손님들을 위해 끝까지 최선을 다해 드실수 있도록...,2021-08-16,0,,"[#산도롱맨도롱▪️오픈시간:, #성산일출봉맛집, #성산아침식사, #성산맛집, #제주..."
1,(협찬)애월바다와 선셋🌅감성까지 더 한스페셜 카레맛집!😋다양한 카레와 토핑들로특별함...,2021-08-17,0,,"[#사이카레@saicurry_aewolOPEN, #사이카레, #구엄리맛집, #애월맛..."
2,#광고 제주도민도 잘 모르는 99.9% 로컬맛집⠀⠀⠀⠀⠀⠀⠀#함덕리 에 위치한 찐+...,2021-08-16,0,제주도 함덕해수욕장,"[#광고, #함덕리, #세화갈비, #갈비찜, #제주도, #제주도여행, #함덕해수욕장..."
3,<광고>#제주분식 https://jejubunsig.modoo.at/ 📌제주시 귀아...,2021-08-16,0,,"[#제주분식, #제주분식⠀간편하게, #모당모당, #돔베라면돔베김밥, #제주맛집, #..."
4,그 유명한 #연돈 에 다녀왔습니다. 그런데ㅎ;;;⠀⠀⠀⠀⠀⠀⠀어마무시한 웨이팅으로 ...,2021-08-18,0,연돈,"[#연돈, #제주도연돈, #테이블링, #등심까스, #수제볼카츠, #아토짱_제주, #..."


### 5.1.8 여러 엑셀 파일의 중복을 제거한 후 통합 저장
- 제주도맛집, 제주도관광, 제주맛집, 제주여행 으로 검색

In [21]:
# 예제 5-12 여러 개의 저장파일 통합하기
jeju_insta_df = pd.DataFrame( [ ] )

folder = './files/'
f_list = ['1_crawling_jejudoMatJip.xlsx', '1_crawling_jejudoGwanGwang.xlsx', '1_crawling_jejuMatJip.xlsx', '1_crawling_jejuYeoHang.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns =['content','data','like','place','tags']


In [22]:
# 예제 5-13 중복 데이터 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)
#jeju_insta_df.to_excel('./files/1_crawling_raw.xlsx', index = False)
jeju_insta_df.to_csv('제주맛집.csv', index=False)

In [23]:
jeju_insta_df.head()

,content,data,like,place,tags
0,제주도 도착~~공항근처 #제주분식 에서 먹은 첫식사만족만족!#제주맛집#제주도맛집#제...,2019-01-01,705,NaN,"['#제주분식', '#제주맛집', '#제주도맛집', '#제주맛집추천', '#제주도맛..."
1,﻿#함덕맛집 무거버거 바다 보면서 먹으니까 더 꿀맛👍🏻 #제주도카페투어#제주일상#함...,2019-01-01,384,NaN,"['#함덕맛집', '#제주도카페투어', '#제주일상', '#함덕', '#jejudo..."
2,.대만족 제주스!.#동문시장#제주동문시장#제주도기념품#제주기념품#제주스는완벽#제주도...,2019-01-01,595,NaN,"['#동문시장', '#제주동문시장', '#제주도기념품', '#제주기념품', '#제주..."
3,#제주여행#2일차무거버거 제주와서 먹은것중에 제일 맛있었음!#제주도여행#제주맛집#함...,2019-01-01,412,NaN,"['#제주여행', '#2일차무거버거', '#제주도여행', '#제주맛집', '#함덕맛..."
4,"🥇1월 영업 안내🥇-1월 2일, 3일(수,목)은 쉬어갑니다-1월 4일,5일,6일(금...",2019-01-01,364,로타리과자점,[]


In [24]:
jeju_insta_df.tail()

,content,data,like,place,tags
1702,Now it's purple😆💜수색 넘 이쀼리 🤭 올해는 아퓨지말구 즐거운 일만 가...,2019-01-01,42,NaN,"['#일상', '#구기자', '#티타임', '#2019', '#새해', '#차스타그..."
1703,.간다간다간다#제주 #친정여행 #겨울방학 #제주여행앞머리는 괜히 전날 잘라서 또순이...,2019-01-01,50,NaN,"['#제주', '#친정여행', '#겨울방학', '#제주여행앞머리는', '#친정찬스'..."
1705,2019년 모두 복 많이많이 받고 풍요로운 한해 보내길 ♥️#사진좀찍으시는님덕에건져...,2019-01-01,68,Jeju,"['#사진좀찍으시는님덕에건져또', '#이순간']"
1706,2018.12.25 ⠀룸에 의자가 두개 밖에 없어서 ⠀이현이는 유모차에 앉아서 식사...,2019-01-01,70,NaN,[]
1707,.2019년 첫날디스이즈핫에서 새해를 맞이하시는 손님들께 떡국을 준비했습니다.한 해...,2019-01-01,48,디스이즈핫,['#HappyNewYearEveryone']


In [25]:
driver.close()